# How-to

This notebook serves as a practical guide to common questions users might have.

In [ ]:
import MDAnalysis as mda
import prolif as plf

In [ ]:
u = mda.Universe(plf.datafiles.TOP, plf.datafiles.TRAJ)
lig = u.select_atoms("resname LIG")
prot = u.select_atoms("protein")
lmol = plf.Molecule.from_mda(lig)
pmol = plf.Molecule.from_mda(prot)

## 1. Changing the parameters for an interaction

In this example, we'll redefine the hydrophobic interaction with a shorter distance.

You have the choice between overwriting the original hydrophobic interaction with the new one, or giving it an original name.

Let's start with a test case: with the default parameters, Y109 is interacting with our ligand.

In [ ]:
fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

### 1.a Overwriting the original interaction

You have to define a class that inherits one of the classes listed in the `prolif.interactions` module

In [ ]:
class Hydrophobic(plf.interactions.Hydrophobic):
    def __init__(self):
        super().__init__(distance=4.0)

In [ ]:
fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

The interaction is not detected anymore. You can reset to the default interaction like so:

In [ ]:
class Hydrophobic(plf.interactions.Hydrophobic):
    pass

fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

### 1.b Reparameterizing an interaction with another name

The steps are identical to above, just give the class a different name:

In [ ]:
class CustomHydrophobic(plf.interactions.Hydrophobic):
    def __init__(self):
        super().__init__(distance=4.0)
        
fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

In [ ]:
fp.customhydrophobic(lmol, pmol["TYR109.A"])

In [ ]:
fp = plf.Fingerprint(["Hydrophobic", "CustomHydrophobic"])
fp.bitvector(lmol, pmol["TYR109.A"])

## 2. Writing your own interaction

Before you dive into this section, make sure that there isn't already an interaction that could just be **reparameterized** to do what you want!

For this, the best is to check the section of the documentation corresponding to the `prolif.interactions` module. There are some generic interactions, like the `_Distance` class, if you just need to define two chemical moieties within a certain distance. Both the Hydrophobic, Ionic, and Metallic interactions inherit from this class!

With that being said, there are a few rules that you must respect when writing your own interaction:

* **Inherit the ProLIF Interaction class**
  
This class is located in `prolif.interactions.Interaction`. If for any reason you must inherit from another class, you can also define the `prolif.interactions._InteractionMeta` as a metaclass.

* **Naming convention**

Your class name must not start with `_` or be named `Interaction`. For non-symmetrical interactions, like hydrogen bonds or salt-bridges, the convention used here is to named the class after the function of the ligand. For example, the class `HBDonor` detects if a ligand acts as a hydrogen bond donor, and the class `Cationic` detects if a ligand acts as a cation.

* **Define a `detect` method**

This method takes exactly two positional arguments (and as many named arguments as you need): a ligand Residue or Molecule and a protein Residue or Molecule (in this order).

* **Return value(s) for the `detect` method**

There are two possibilities here, depending on whether or not you want to access the indices of atoms responsible for the interaction. If you don't need this information, just return `True` if the interaction is detected, `False` otherwise. If you need to access atomic indices, you must return the following items in this order: 

  * `True` or `False` for the detection of the interaction
  * The index of the ligand atom, or None if not detected
  * The index of the protein atom, or None if not detected

In [ ]:
from scipy.spatial import distance_matrix

# without atom indices
class CloseContact(plf.interactions.Interaction):
    def detect(self, res1, res2, threshold=2.0):
        dist_matrix = distance_matrix(res1.xyz, res2.xyz)
        if (dist_matrix <= threshold).any():
            return True
        return False

fp = plf.Fingerprint()
fp.closecontact(lmol, pmol["ASP129.A"])

In [ ]:
# with atom indices
class CloseContact(plf.interactions.Interaction):
    def detect(self, res1, res2, threshold=2.0):
        dist_matrix = distance_matrix(res1.xyz, res2.xyz)
        mask = dist_matrix <= threshold
        if mask.any():
            res1_i, res2_i = np.where(mask)
            # return the first solution
            return True, res1_i[0], res2_i[0]
        return False, None, None

fp = plf.Fingerprint()
fp.closecontact(lmol, pmol["ASP129.A"])

By default, the fingerprint will modify all interaction classes to only return the boolean value. To get the atom indices when using your custom class, you must choose one of the following options:

* Use the `__wrapped__` argument when calling the class as a fingerprint method:

In [ ]:
fp.closecontact.__wrapped__(lmol, pmol["ASP129.A"])

* Use the `bitvector_atoms` method instead of `bitvector`:

In [ ]:
fp = plf.Fingerprint(["CloseContact"])
bv, indices = fp.bitvector_atoms(lmol, pmol["ASP129.A"])
bv, indices

* Use the `return_atoms=True` argument when calling the `run` method:

In [ ]:
fp.run(u.trajectory[0:1], lig, prot, return_atoms=True)
fp.ifp

* Directly use your class:

In [ ]:
cc = CloseContact()
cc.detect(lmol, pmol["ASP129.A"])

## 3. Ignoring the protein backbone when computing interactions

In [ ]:
# not implemented yet